In [1]:
import graph_tool as gt
import numpy as np
from graph_tool import collection,inference,generation
import matplotlib.pyplot as plt
import random

In [2]:
G = gt.load_graph_from_csv('ants_data.csv',eprop_types=["int","int"],
                           eprop_names=["StartTime","EndTime"],skip_first=True)
G.num_edges()
G.reindex_edges

<bound method Graph.reindex_edges of <Graph object, undirected, with 89 vertices and 1911 edges, 1 internal vertex property, 2 internal edge properties, at 0x7f4e09035160>>

In [3]:
def is_missing(G,i,j):
    '''If i,j are linked in graph G, return false, otherwise true.'''
    if G.edge(i,j) or G.edge(j,i):
        return False
    else:
        return True
    
def missing_edge(G):
    '''Return a list of all missing links in G.'''
    n = G.num_vertices()
    new_edge = []
    for i in range(n):
        for j in range(i+1,n):
            if is_missing(G,i,j):
                new_edge.append((i,j))
    # new_edge = [[i,j] for i in range(n) for j in range(i+1,n) if [i,j] not in edges]
    return new_edge

def simplify(G):
    '''Remove multiple links in Grpah G.'''
    H = G.copy()
    H.clear_edges()
    for e in G.edges():
        if is_missing(H,e.source(),e.target()):
            if e.source() < e.target():
                H.add_edge(e.source(),e.target())
            else:
                H.add_edge(e.target(),e.source())
    return H

In [4]:
def missing_edge_prob(G):
    '''Return the probability of all missing links being added.'''
    print(1)
    missing_edges = missing_edge(G)
    state = gt.inference.minimize_nested_blockmodel_dl(G,state_args=dict(deg_corr=False))
    probs = [[] for b in range(len(missing_edges))]
    def collect_edge_probs(s):
        for i in range(len(missing_edges)):
            p = s.get_edges_prob([missing_edges[i]], entropy_args=dict(partition_dl=False))
            probs[i].append(p)
    gt.inference.mcmc_equilibrate(state, force_niter=800, mcmc_args=dict(niter=10),
                    callback=collect_edge_probs)
    def get_avg(p):
        p = np.array(p)
        pmax = p.max()
        p -= pmax
        return pmax + np.log(np.exp(p).mean())
    pn =  [get_avg(probs[n]) for n in range(len(missing_edges))]
    p_sum = get_avg(pn) + np.log(len(pn))
    return {missing_edges[i]:np.exp(pn[i]-p_sum) for i in range(len(pn))}

In [5]:
def AUC(G,sample_number,probe_portion=0.1,precision=False):
    '''AUC of SBM and if precision is True, precsion will also be computed and returned. 
       Arguments are similar to AUC implemented in networkx.'''
    G.reindex_edges()
    simpH = simplify(G)
    edgelist = simpH.get_edges()
    n = simpH.num_vertices()
    ne = simpH.num_edges()
    edgelistG = G.get_edges()
    p = int(probe_portion*ne)
    q = p//2
    H = gt.Graph()
    Hprobelinks = random.sample(list(edgelist),p)
    #print(len(Hprobelinks))
    #print(Hprobelinks)
    remove_index = []
    for a in Hprobelinks:
        tobe_removed_edges = G.edge(a[0],a[1],all_edges=True) + G.edge(a[1],a[0],all_edges=True)
        remove_index += [G.edge_index[e] for e in tobe_removed_edges]
    edgelistG = np.delete(edgelistG,remove_index,0)
    H.add_edge_list(edgelistG)
    B = missing_edge_prob(H)
    rank_list = list(B.values())
    rank_list.sort(reverse=True)
    Hpotential_links = missing_edge(G)
    auc = 0
    pre_list = np.zeros(9)
    for m in range(sample_number):
        probe_sample = random.sample(Hprobelinks,q)
        potential_sample = random.sample(Hpotential_links,q)
        m1,m2 = 0,0
        L = 0
        for i in range(q):
            e1 = probe_sample[i]
            e2 = potential_sample[i]
            if B[e1[0],e1[1]] > B[e2[0],e2[1]]:
                m1 += 1
            elif B[e1[0],e1[1]] == B[e2[0],e2[1]]:
                m2 += 1
        auc += (m1 + 0.5*m2)/q
    if precision:
        L_list = np.array([len([e for e in Hprobelinks if B[e[0],e[1]] >= rank_list[int(p*tp)]]) 
                            for tp in np.arange(0.1,1,0.1)])
        r_list = np.array([int(p*tp) for tp in np.arange(0.1,1,0.1)])
        pre_list += L_list/r_list
        return auc/sample_number,pre_list
    else:
        return auc/sample_number

In [ ]:
auc1 = [AUC(G,20,precision=True) for n in range(5)]
np.save("ant_ndcauc1",auc1)

1


In [ ]:
auc2 = [AUC(G,20,probe_portion=0.3,precision=True) for n in range(5)]
np.save("ant_ndcauc2",auc2)

In [ ]:
print(auc1)

In [ ]:
auc3 = [AUC(G,20,probe_portion=0.5,precision=True) for n in range(5)]
np.save("ant_ndcauc3",auc3)

In [ ]:
print(auc2)

In [ ]:
auc4 = [AUC(G,20,probe_portion=0.9,precision=True) for n in range(5)]
np.save("ant_ndcauc4",auc4)

In [ ]:
print(auc3)

In [ ]:
print(auc4)